In [1]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from collections import Counter
from tqdm import tqdm
import pandas as pd

In [2]:
df = pd.read_csv('training_freek_feateng.csv')

In [3]:
columns = ['srch_id', 'booking_bool', 'click_bool', 'position', 'random_bool', 'promotion_flag', 'srch_length_of_stay', 'prop_id']
df = df[columns].copy()

In [4]:
df.head()

,srch_id,booking_bool,click_bool,position,random_bool,promotion_flag,srch_length_of_stay,prop_id
0,1,0,0,27,1,0,1,893
1,1,0,0,26,1,0,1,10404
2,1,0,0,21,1,0,1,21315
3,1,0,0,34,1,0,1,27348
4,1,0,0,4,1,0,1,29604


In [5]:
X = df.drop(['booking_bool','position','prop_id'], axis=1)
y = df['booking_bool']

In [6]:
df.columns

Index(['srch_id', 'booking_bool', 'click_bool', 'position', 'random_bool',
       'promotion_flag', 'srch_length_of_stay', 'prop_id'],
      dtype='object')

In [7]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.001)

In [8]:
# save click_bool and srch_id from test
click_list = X_test['click_bool']
srchid_list = X_test['srch_id']


# remove click_bool and srch_id from train and test
X_train = X_train.drop(['click_bool', 'srch_id'], axis=1)
X_test = X_test.drop(['click_bool', 'srch_id'], axis=1)

In [9]:
X_train.columns

Index(['random_bool', 'promotion_flag', 'srch_length_of_stay'], dtype='object')

In [10]:
# Scale the features using StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [11]:
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [123]:
y_pred = []
for i in tqdm(range(len(X_test))):
    pred = knn.predict_proba([X_test[i]])
    y_pred.append(pred)

100%|██████████| 4959/4959 [00:21<00:00, 231.10it/s]


In [138]:
print(y_pred)

[array([[1., 0.]]), array([[1., 0.]]), array([[1., 0.]]), array([[1., 0.]]), array([[1., 0.]]), array([[1., 0.]]), array([[1., 0.]]), array([[1., 0.]]), array([[1., 0.]]), array([[1., 0.]]), array([[1., 0.]]), array([[1., 0.]]), array([[0.66666667, 0.33333333]]), array([[1., 0.]]), array([[1., 0.]]), array([[1., 0.]]), array([[1., 0.]]), array([[1., 0.]]), array([[1., 0.]]), array([[1., 0.]]), array([[1., 0.]]), array([[1., 0.]]), array([[1., 0.]]), array([[1., 0.]]), array([[1., 0.]]), array([[1., 0.]]), array([[1., 0.]]), array([[1., 0.]]), array([[1., 0.]]), array([[0.66666667, 0.33333333]]), array([[1., 0.]]), array([[1., 0.]]), array([[1., 0.]]), array([[1., 0.]]), array([[1., 0.]]), array([[1., 0.]]), array([[1., 0.]]), array([[1., 0.]]), array([[1., 0.]]), array([[1., 0.]]), array([[1., 0.]]), array([[1., 0.]]), array([[1., 0.]]), array([[1., 0.]]), array([[1., 0.]]), array([[1., 0.]]), array([[1., 0.]]), array([[1., 0.]]), array([[1., 0.]]), array([[1., 0.]]), array([[1., 0.]])

In [139]:
# Count the occurrences of each element
counts = Counter(y_pred)

# Print the counts
print("Count of 0's:", counts[0])
print("Count of 1's:", counts[1])

TypeError: unhashable type: 'numpy.ndarray'

# Predicting on test

In [16]:
test_df =  pd.read_csv('test_freek_feateng.csv')

In [17]:
columns = ['srch_id', 'random_bool', 'promotion_flag', 'srch_length_of_stay', 'prop_id']

test_df = test_df[columns].copy()

In [18]:
test_df.columns

Index(['srch_id', 'random_bool', 'promotion_flag', 'srch_length_of_stay',
       'prop_id'],
      dtype='object')

In [19]:
prop_ids = test_df['prop_id']
srch_ids = test_df['srch_id']
test_df = test_df.drop(['prop_id', 'srch_id'],axis=1)

y_pred = knn.predict_proba(test_df)

prediction_df = pd.concat( [srch_ids, prop_ids] , axis=1)
prediction_df['prob'] = y_pred[:,1]
prediction_df = prediction_df.groupby('srch_id').apply(lambda x: x.sort_values('prob', ascending=False))

# prediction_df = prediction_df.reset_index(drop=True)
prediction_df = prediction_df.drop(['prob'], axis=1)

prediction_df.to_csv('rf_predictions.csv', index=False)

/Users/freekcool/miniforge3/envs/env_tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  warnings.warn(


In [20]:
prediction_df = pd.concat( [srch_ids, prop_ids] , axis=1)

In [101]:
prediction_df

,srch_id,prop_id
0,1,3180
1,1,5543
2,1,14142
3,1,22393
4,1,24194
...,...,...
49995,3261,73456
49996,3261,84390
49997,3261,87468
49998,3261,93466


In [106]:
prediction_df['prob'] = y_pred[:,1]

TypeError: list indices must be integers or slices, not tuple